In [1]:
import torch
import lightgbm as lgb
from tqdm import tqdm
from loaders.lending_loader import load_data, mono_list
from monotonenorm import SigmaNet, GroupSort, direct_norm
from sklearn.metrics import balanced_accuracy_score, accuracy_score
import numpy as np

In [2]:
Xtr, Ytr, Xts, Yts = load_data(get_categorical_info=False)
monotonic_constraints = [int(i in mono_list) for i in range(Xtr.shape[1])]

In [6]:
clf = lgb.LGBMRegressor(n_estimators=10000, max_depth=5, learning_rate=.1, monotone_constraint=monotonic_constraints)
clf.fit(Xtr, Ytr, early_stopping_rounds=200, eval_set=[(Xts, Yts)], eval_metric='mse', verbose=0)

LGBMRegressor(max_depth=5,
              monotone_constraint=[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              n_estimators=10000)

In [7]:
acc, bacc = 0,0
for i in np.linspace(0,1,50):
  acc = max(acc, accuracy_score(Yts, clf.predict(Xts)>i))
  bacc = max(bacc, balanced_accuracy_score(Yts, clf.predict(Xts)>i))
acc, bacc

(0.6563835241839002, 0.6567178915278866)

In [45]:

per_layer_lip = 2
width = 64

from monotonenorm import direct_norm
activation = lambda : GroupSort(2)

direct_norm = lambda x, *args, **kwargs: x # make it a normal network
activation = lambda : torch.nn.ReLU()


model = torch.nn.Sequential(
  direct_norm(torch.nn.Linear(Xtr.shape[1], width), kind="one-inf", alpha=per_layer_lip),
  activation(),
  direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
  activation(),
  direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
  activation(),
  direct_norm(torch.nn.Linear(width, 1), kind="inf", alpha=per_layer_lip),
).cuda()
#model = SigmaNet(model, sigma=per_layer_lip**4, monotone_constraints=[int(i in mono_list) for i in range(Xtr.shape[1])]).cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
print('params:', sum(p.numel() for p in model.parameters()))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.999)


params: 10241


In [46]:
print(model)
Xtrt = torch.tensor(Xtr, dtype=torch.float32).cuda()
Ytrt = torch.tensor(Ytr, dtype=torch.float32).view(-1, 1).cuda()
Xtst = torch.tensor(Xts, dtype=torch.float32).cuda()
Ytst = torch.tensor(Yts, dtype=torch.float32).view(-1, 1).cuda()

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Xtrt, Ytrt), batch_size=int(2**14), shuffle=True)

bar = tqdm(range(1000))
for i in bar:
  for Xi, yi in dataloader:
    y_pred = model(Xi)
    losstr = torch.nn.functional.binary_cross_entropy_with_logits(y_pred, yi)
    optimizer.zero_grad()
    losstr.backward()
    optimizer.step()
    scheduler.step()

  with torch.no_grad():
    y_predts = model(Xtst)
    lossts = torch.nn.functional.binary_cross_entropy_with_logits(y_predts, Ytst)
    acc = 0
    for i in np.linspace(.3, .7, 20):
      acc = max(acc, accuracy_score(Ytst.cpu().numpy(), y_predts.cpu().numpy()>i))
    bar.set_description(f'Loss: {losstr.item():.4f} {lossts.item():.4f}, acc: {acc.item():.4f}')

Sequential(
  (0): Linear(in_features=28, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=1, bias=True)
)


Loss: 0.6236 0.6244, acc: 0.6352:   1%|          | 7/1000 [00:24<58:53,  3.56s/it]


KeyboardInterrupt: 